In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-11-06 01:11:57--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4350 (4.2K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   4.25K  --.-KB/s    in 0s      

2025-11-06 01:11:57 (31.2 MB/s) - ‘minsearch.py.2’ saved [4350/4350]



In [4]:
import minsearch

/workspaces/llm-zoomcamp/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [5]:
from dotenv import load_dotenv; load_dotenv()

True

In [6]:
import json

In [7]:
with open('documents.json', 'rt') as f_in: 
    doc_raw = json.load(f_in)

In [9]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
index = minsearch.Index(
    text_fields={"question", "text", "section"},
    keyword_fields={"course"}
)

In [11]:
index.fit(documents)

In [14]:
from openai import OpenAI

In [15]:
client = OpenAI()

In [38]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    result = index.search(
        query=query,
        filter_dict={'course': 'machine-learning-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return result


In [43]:
def build_prompt(query, search_result):
    prompt_template = """
Eres un asistente de cursos. Responde la pregunta basandote en el CONTEXTO. Solo usa los factos del CONTEXTO para responder la PREGUNTA. 
Si no existe CONTEXTO para responder la PREGUNTA sólo di NONE.

PREGUNTA: {question}

CONTEXTO: {context}
""".strip()
    
    context = ""
    
    for doc in search_result:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nawnser: {doc['text']}\n\n".strip()

    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [46]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-5-nano',
    messages=[{"role": "user", "content": prompt}]
    )

    response = response.choices[0].message.content
    return response

In [54]:
def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    anwser = llm(prompt)
    return anwser

In [55]:
rag('the course already started. Can I still enroll?')

'Sí, puedes unirte aunque el curso ya haya empezado. No podrás entregar algunos de los deberes, pero aún puedes participar en el curso. Para obtener un certificado, debes entregar 2 de los 3 proyectos y revisar 3 proyectos de tus compañeros antes de la fecha límite. Por ejemplo, si te unes a finales de noviembre y trabajas en dos proyectos, seguirás siendo elegible para el certificado.'

In [56]:
from elasticsearch import Elasticsearch

In [59]:
es_client = Elasticsearch('http://localhost:9200')

In [62]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

In [63]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [64]:
from tqdm.auto import tqdm

In [66]:
for doc in tqdm(documents): 
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [73]:
def elastic_search(query): 
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    search_response = es_client.search(index=index_name, body=search_query)
    result_search = []

    for hit in search_response['hits']['hits']:
                result_search.append(hit['_source'])
    return result_search    

In [74]:
def rag_es(query):
    search_result = elastic_search(query)
    prompt = build_prompt(query, search_result)
    anwser = llm(prompt)
    return anwser

In [75]:
rag('the course already started. Can I still enroll?')

'Sí, puedes. No podrás entregar algunas de las tareas, pero aún puedes participar en el curso. Para obtener el certificado, debes entregar 2 de 3 proyectos y revisar 3 proyectos de tus compañeros antes de la fecha límite. Esto significa que si te unes al curso a finales de noviembre y trabajas en dos proyectos, seguirás siendo elegible para un certificado.'